In [1]:
import pandas as pd
import os
import requests
import numpy as np 
import io
import tqdm
import sys 
from utils.files import *

print('Current working directory:', os.getcwd())

Current working directory: /home/prichter/Documents/find-a-bug/sandbox


In [8]:
LOG_DIR = os.path.join(os.getcwd(), '../scripts/log/')

# Running into errors uploading certain annotations to the MariaDB database. 
failures_df = []
for file_name in os.listdir(LOG_DIR):
    failures_df.append(pd.read_csv(os.path.join(LOG_DIR, file_name), comment='#'))
failures_df = pd.concat(failures_df)

/tmp/ipykernel_468266/888517050.py:6: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  failures_df.append(pd.read_csv(os.path.join(LOG_DIR, file_name), comment='#'))
/tmp/ipykernel_468266/888517050.py:6: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  failures_df.append(pd.read_csv(os.path.join(LOG_DIR, file_name), comment='#'))


In [12]:
print('Number of gene IDs in failures_df:', len(failures_df))
print('Number of gene IDs after de-duplicating:', len(failures_df.gene_id.unique()))
failures_df['duplicated_gene_id'] = failures_df.gene_id.duplicated()
# Ah it makes sense that these are duplicated because they are for annotations, and there can be multiple annotations per gene. 
# print(failures_df[failures_df.duplicated_gene_id])

Number of gene IDs in failures_df: 1616073
Number of gene IDs after de-duplicating: 1546186


In [12]:
print(len(failures_df.genome_id.unique()), 'genome IDs failed to upload.')
# Perhaps I should first make sure the files are actually present in the source directory. 
# UPDATE: They all are! So maybe the genes aren't in the files?

1100 genome IDs failed to upload.


In [3]:
failing_genome_ids = failures_df['genome_id'].unique()
np.savetxt('failing_genome_ids.txt', failing_genome_ids, fmt='%s')

In [4]:
gene_ids_in_database_df = []
for genome_id in failing_genome_ids:
    url = f'http://microbes.gps.caltech.edu:8000/get/proteins_r207?genome_id[eq]{genome_id}'
    result = requests.get(url).text 
    try:
        gene_ids_in_database_df.append(pd.read_csv(io.StringIO(result)))
    except:
        print(f'There doesnt seem to be any results for {genome_id}. Length of results:', len(result))
gene_ids_in_database_df = pd.concat(gene_ids_in_database_df)
gene_ids_in_database_df.set_index('gene_id').to_csv('gene_ids_in_database.csv')

    

There doesnt seem to be any results for GCA_011364235.1. Length of results: 0
There doesnt seem to be any results for GCF_015245355.1. Length of results: 0
There doesnt seem to be any results for GCA_011364155.1. Length of results: 0
There doesnt seem to be any results for GCA_011364265.1. Length of results: 0
There doesnt seem to be any results for GCF_015244745.1. Length of results: 0
There doesnt seem to be any results for GCA_011364225.1. Length of results: 0
There doesnt seem to be any results for GCA_011364305.1. Length of results: 0


In [ ]:
# gene_ids_not_in_database_already_downloaded = np.loadtxt('gene_ids_not_in_database.txt', dtype=str)
# gene_ids_to_check = [gene_id for gene_id in failures_df.gene_id.values if (gene_id not in gene_ids_not_in_database_already_downloaded)]
# print(f'Already confirmed {len(gene_ids_not_in_database_already_downloaded)} missing genes.')

gene_ids_not_in_database = []

chunk_size = 100
# Why are there duplicate gene IDs? Are some of the IDs not unique?
gene_ids_to_check = failures_df.gene_id.unique().tolist()
gene_ids_to_check = [gene_ids_to_check[i * chunk_size:(i + 1) * chunk_size] for i in range(len(gene_ids_to_check) // chunk_size + 1)]

pbar = tqdm.tqdm(gene_ids_to_check, desc='Querying Find-A-Bug... Found 0 missing genes.')
for chunk in pbar:
    filter_ = 'gene_id[eq]' + '[or]'.join(chunk)
    url = f'http://microbes.gps.caltech.edu:8000/get/proteins_r207?' + filter_

    result = requests.get(url).text 

    if len(result) == 0:
        print('No results returned!')
        gene_ids_not_in_database += chunk
        pbar.set_description(f'Querying Find-A-Bug... Found {len(gene_ids_not_in_database)} missing genes.')
    else:
        result_df = pd.read_csv(io.StringIO(result))
        if len(result_df) < len(chunk):
            print(f'Retrieved {len(result_df)} results, expected {len(chunk)}.')
            gene_ids_not_in_database += [gene_id for gene_id in chunk if (gene_id not in result_df.gene_id)]
            pbar.set_description(f'Querying Find-A-Bug... Found {len(gene_ids_not_in_database)} missing genes.')

np.savetxt('gene_ids_not_in_database.txt', gene_ids_not_in_database, fmt='%s')


In [70]:
failures_df = failures_df[failures_df.gene_id.isin(gene_ids_not_in_database)]

In [73]:
print('Number of genomes with failed genes:', len(failures_df.genome_id.unique()))

Number of genomes with failed genes: 28


In [5]:
genome_ids = [f"\'{genome_id}\'" for genome_id in failures_df.genome_id.unique()]
print(f"[{', '.join(genome_ids)}]")

print('GCA_011364155.1' in genome_ids)

['GCF_900112355.1', 'GCA_016708875.1', 'GCA_903892355.1', 'GCF_014926385.1', 'GCA_017452545.1', 'GCA_002928875.1', 'GCF_008271465.1', 'GCA_003650545.1', 'GCF_000744705.1', 'GCF_003752025.1', 'GCF_001583495.1', 'GCA_015234465.1', 'GCA_003151935.1', 'GCA_902789265.1', 'GCA_014529615.1', 'GCA_018829985.1', 'GCA_017993235.1', 'GCF_000424285.1', 'GCA_016870515.1', 'GCF_017922435.1', 'GCF_008802355.1', 'GCA_011338225.1', 'GCA_019113455.1', 'GCA_015066445.1', 'GCF_000272105.2', 'GCF_000746085.1', 'GCA_905234465.1', 'GCA_900552055.1', 'GCA_003150135.1', 'GCA_018715585.1', 'GCA_008080615.1', 'GCA_009840625.1', 'GCA_017408325.1', 'GCF_004331035.1', 'GCA_902826575.1', 'GCF_002288305.1', 'GCF_008932295.1', 'GCF_011319465.1', 'GCA_018829445.1', 'GCF_000336995.1', 'GCA_017557185.1', 'GCA_016938695.1', 'GCA_001805885.1', 'GCA_001821435.1', 'GCA_002789815.1', 'GCA_016926395.1', 'GCF_000585495.1', 'GCA_012514185.1', 'GCF_001590965.1', 'GCA_017456765.1', 'GCA_903835865.1', 'GCA_013820755.1', 'GCF_013283

In [13]:
# Ok, so it seems as though there are entire genomes missing in the proteins_r207 table, as well as some genes
# for genomes which are included. 

genes_df = []
# I dowloaded all of the FASTA files for the genomes with missing genes to see what is going on. 
for file_name in os.listdir('./genome_ids_with_missing_genes'):
    path = os.path.join('./genome_ids_with_missing_genes', file_name)
    file = ProteinsFile(path)
    genes_df += file.entries()
genes_df = pd.DataFrame(genes_df)
    

In [29]:
with open('./gene_ids_not_in_database.txt', 'r') as f:
    gene_ids_not_in_database = f.readlines()
# Why is np.loadtxt not working?
# genes_ids_not_in_database = np.loadtxt('/home/prichter/Documents/find-a-bug/sandbox/gene_ids_not_in_database.txt', dtype=str)
print(len(gene_ids_not_in_database), 'gene IDs not in the database.')

df = genes_df[genes_df.gene_id.isin(genes_ids_not_in_database)]
print(len(df), 'genes not in the database, but are in the source files.')
# Ok, so have confirmed that the issue is not due to the fact that the genes are missing in the source files. 
# There also doesn't seem to be anything particularly strange about the dropped entries. I wonder what happened?

9200 gene IDs not in the database.
9200 genes not in the database, but are in the source files.


In [30]:
df

,gene_id,start,stop,strand,scaffold_id,partial,rbs_motif,gc_content,seq,genome_id,version
3889,NZ_JADQDN010000009.1_40,45606,49127,-,21,00,None,0.642,MKPVLTRALDALKMGQSLTLSGVPDGFDALAVADLARGLSGQVEGP...,GCF_015694425.1,None
3890,NZ_JADQDN010000009.1_41,49205,49501,-,21,00,GGAG/GAGG,0.636,MSGTTRSSAGLDTRRRQILYRSWHRGMREMDLIMGRFADAEIGELS...,GCF_015694425.1,None
3891,NZ_JADQDN010000009.1_42,49591,51708,+,21,00,None,0.645,MSLRPSILDPLFAPANTLPGVGPKIAPLLDRLLGEAGKPTRVADLL...,GCF_015694425.1,None
3892,NZ_JADQDN010000009.1_43,51881,53707,-,21,00,AGGA/GGAG/GAGG,0.606,MCGIVGIIGKAPVAPQIVEALKRLEYRGYDSAGVATLEEGRLDRRR...,GCF_015694425.1,None
3893,NZ_JADQDN010000009.1_44,53818,55113,-,21,00,GGAG/GAGG,0.620,MKSAKPKVLHQVASRSMLGHVMATLAQAGATSTAVVIGPDREDVAK...,GCF_015694425.1,None
...,...,...,...,...,...,...,...,...,...,...,...
76402,DTBN01000011.1_3,1587,2666,+,12,00,None,0.456,MIKMCAIHDNVKRGISKMAVVLIVALIIVVGIVAWYAFSSIGVAPQ...,GCA_011364235.1,None
76403,DTBN01000011.1_4,2713,4233,+,12,00,GGTG,0.420,MLVVELRKITKVYPGGIVANYEVDFDLRKGEVHALLGENGAGKTTL...,GCA_011364235.1,None
76404,DTBN01000011.1_5,4298,5266,+,12,00,None,0.465,MGKDPLQAYYWLFQGAFSPSAISETFVRATPLLLISIGLAVVFRAK...,GCA_011364235.1,None
76405,DTBN01000011.1_6,5263,6204,+,12,00,GGT,0.445,MITAEVILGLLYSSITLLTPLLLAGLAELLVERSGILNLSLEGTML...,GCA_011364235.1,None
